# 프로젝트: 멋진 챗봇 만들기

- **챗봇과 번역기는 같은 집안**
- 앞서 배운 Seq2seq 번역기와 Transformer번역기에 적용할 수도 있겠지만, 이번 노드에서 배운 번역기 성능 측정법을 챗봇에도 적용해 보기

## 목차
- 1. 데이터 다운로드
- 2. 데이터 정제
- 3. 데이터 토큰화
- 4. Augmentation
- 5. 데이터 벡터화
- 6. 훈련하기
- 7. 성능 측정하기

## 루브릭
|평가문항|상세기준|
|---|---|
|1. 챗봇 훈련데이터 전처리 과정이 체계적으로 진행되었는가|챗봇 훈련데이터를 위한 전처리와 augmentation이 적절히 수행되어 3만개 가량의 훈련데이터셋이 구축됨|
|2. transformer 모델을 활용한 챗봇 모델이 과적합을 피해 안정적으로 훈련되었는가|과적합을 피할 수 있는 하이퍼파라미터 셋이 적절히 제시됨|
|3. 챗봇이 사용자의 질문에 그럴듯한 형태로 답하는 사례가 있는가|주어진 예문을 포함하여 챗봇에 던진 질문에 적절히 답하는 사례가 제출됨|

## 1. 데이터 다운로드

- 아래 링크에서 'ChatbotData.csv'를 다운로드해 챗봇 훈련 데이터 확보
- 'csv' 파일을 읽는 데에는 'pandas'라이브러리가 적합
- 읽어 온 데이터의 질문과 답변을 각각 'questions', 'answers' 변수에 나눠서 저장
<br>

- [songs/Chatbot_data](https://github.com/songys/Chatbot_data)

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

import re
import os
import io
import time
import random

import gensim
from collections import Counter
from konlpy.tag import Mecab

from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('./data/ChatbotData.csv')
data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [3]:
questions = data['Q']
answers = data['A']

In [4]:
print(len(questions), len(answers))
print('--------------------------')
print(questions.head())
print('--------------------------')
print(answers.head())

11823 11823
--------------------------
0             12시 땡!
1        1지망 학교 떨어졌어
2       3박4일 놀러가고 싶다
3    3박4일 정도 놀러가고 싶다
4            PPL 심하네
Name: Q, dtype: object
--------------------------
0     하루가 또 가네요.
1      위로해 드립니다.
2    여행은 언제나 좋죠.
3    여행은 언제나 좋죠.
4     눈살이 찌푸려지죠.
Name: A, dtype: object


## 2. 데이터 정제

- 아래 조건을 만족하는 'preprocess_sentence()'함수를 구현
    - 1. 영문자의 경우, **모두 소문자로 변환**
    - 2. 영문자와 한글, 숫자, 그리고 주요 특수문자를 제외하곤 **정규식을 활용하여 모두 제거**
- *문장부호 양옆에 공백을 추가하는 등 이전과 다르게 생략된 기능들은 우리가 사용할 토크나이저가 지원하기 때문에 굳이 구현하지 않아도 괜찮음*

In [5]:
def preprocess_sentence(sentence):
    sentence = sentence.lower()
    sentence = re.sub(r"[^a-zA-Z가-힣0-9?.!,]+", " ", sentence)
    return sentence

## 3. 데이터 토큰화

- 토큰화에는 *KoNLPy*의 'mecab' 클래스 사용하기
- 아래 조건을 만족하는 'build_corpus()' 함수 구현하기
    - 1. **소스 문장 데이터**와 **타겟 문장 데이터**를 입력으로 받기
    - 2. 데이터를 앞서 정의한 'preprocess_sentences()'함수로 **정제하고, 토큰화**하기
    - 3. 토큰화는 **전달받은 토크나이즈 함수를 사용**. 이번엔 'mecab.morphs' 함수를 전달하면 됨
    - 4. 토큰의 개수가 일정 길이 이상인 문장은 **데이터에서 제외**
    - 5. **중복되는 문장은 데이터에서 제외**. '소스: 타겟' 쌍을 비교하지 않고 소스는 소스대로 타겟은 타겟대로 검사. 중복 쌍이 흐트러지지 않도록 유의
<br>

- 구현한 함수를 활용하여 'questions'와 'answers'를 각각 'que_corpus', 'ans_corpus'에 토큰화하여 저장

In [6]:
mecab = Mecab()

In [7]:
def build_corpus(src_data, tgt_data):
    mecab_src_corpus, mecab_tgt_corpus = [], []
    mecab_src_len_list, mecab_tgt_len_list = [], []
    
    for s, t in zip(src_data, tgt_data):
        s = mecab.morphs(preprocess_sentence(s))
        t = mecab.morphs(preprocess_sentence(t))
        
        mecab_src_corpus.append(s)
        mecab_tgt_corpus.append(t)
        
        mecab_src_len_list.append(len(s))
        mecab_tgt_len_list.append(len(t))

    mecab_num_tokens = mecab_src_len_list + mecab_tgt_len_list
    
    mean_len = np.mean(mecab_num_tokens)
    max_len = np.max(mecab_num_tokens)
    mid_len = np.median([mean_len, max_len])
    print(f'mid_len : {mid_len}')
    
    src_corpus, tgt_corpus = [], []
    for q, a in zip(mecab_src_corpus, mecab_tgt_corpus):
        if len(q) <= mid_len and len(a) <= mid_len:
            if q not in src_corpus and a not in tgt_corpus:
                src_corpus.append(q)
                tgt_corpus.append(a)
    
    return src_corpus, tgt_corpus

In [8]:
que_corpus, ans_corpus = build_corpus(questions, answers)

mid_len : 23.849149961938593


In [9]:
que_corpus[:5]

[['12', '시', '땡', '!'],
 ['1', '지망', '학교', '떨어졌', '어'],
 ['3', '박', '4', '일', '놀', '러', '가', '고', '싶', '다'],
 ['ppl', '심하', '네'],
 ['sd', '카드', '망가졌', '어']]

In [10]:
ans_corpus[:5]

[['하루', '가', '또', '가', '네요', '.'],
 ['위로', '해', '드립니다', '.'],
 ['여행', '은', '언제나', '좋', '죠', '.'],
 ['눈살', '이', '찌푸려', '지', '죠', '.'],
 ['다시', '새로', '사', '는', '게', '마음', '편해요', '.']]

In [11]:
print(len(que_corpus))
print(len(ans_corpus))

7637
7637


## 4. Augmentation

- 주어진 데이터는 **1만개 가량으로 적은 편**에 속함
- **Lexical Substitution을 실제로 적용**해 보기
<br>

- 아래 링크를 참고하여 **한국얼오 사전 훈련된 Embedding 모델을 다운로드**
- 'Korean(w)'가 Word2Vec으로 학습한 모델이며 용량도 적당하므로 사이트에서 'Korean(w)'를 찾아 다운로드하고, 'ko.bin' 파일을 얻기
- [Kyubyong/wordvectors](https://github.com/Kyubyong/wordvectors)
<br>

- 다운로드한 모델을 활용해 **데이터를 Augmentation**
- 앞서 정의한 'lexical_sub()'함수를 참고하면 도움이 많이 될 것
<br>

- *Augmentation*된 'que_corpus'와 원본 'ans_corpus'가 병렬을 이루도록, 이후엔 반대로 원본 'que_corpus'와 *Augmentation*된 'ans_corpus'가 병렬을 이루도록 하여 **전체 데이터가 원래의 3배가량으로 늘어나도록** 함

In [12]:
word2vec_path = os.getenv('HOME') + '/aiffel/Going_Deeper(NLP)/NLP_12. 번역기는 대화에만 능하다/data/ko.bin'
word2vec = gensim.models.Word2Vec.load(word2vec_path)

In [13]:
wv = gensim.models.Word2Vec.load('./data/ko.bin')

In [14]:
def lexical_sub(sentence, word2vec):
    import random

    res = ""
    toks = sentence

    try:
        _from = random.choice(toks)
        _to = word2vec.most_similar(_from)[0][0]

    except:   # 단어장에 없는 단어
        return None

    for tok in toks:
        if tok is _from: res += _to + " "
        else: res += tok + " "

    return res

In [15]:
lexical_sub(que_corpus[0], wv)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if __name__ == '__main__':


'12 시 끗 ! '

In [16]:
from tqdm import tqdm_notebook

new_que_corpus = []
new_ans_corpus = []

for idx in tqdm_notebook(range(len(que_corpus))):
    que_augmented = lexical_sub(que_corpus[idx], wv)
    ans = ans_corpus[idx]
    
    if que_augmented is not None:
        new_que_corpus.append(que_augmented.split())
        new_ans_corpus.append(ans)
    else:continue
    
for idx in tqdm_notebook(range(len(ans_corpus))):
    que = que_corpus[idx]
    ans_augmented = lexical_sub(ans_corpus[idx], wv)
    
    if ans_augmented is not None:
        new_que_corpus.append(que)
        new_ans_corpus.append(ans_augmented.split())
    else:continue

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/7637 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if __name__ == '__main__':
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app


  0%|          | 0/7637 [00:00<?, ?it/s]

In [17]:
print(len(new_que_corpus))
print(len(new_ans_corpus))

13244
13244


In [18]:
new_que_corpus[:5]

[['12', '시가', '땡', '!'],
 ['1', '지망', '학교의', '떨어졌', '어'],
 ['sns', '맞', '팔', '왜', '안', '하', '꼼짝'],
 ['가끔', '궁금', '해의'],
 ['가끔', '은', '혼자', '인', '도록', '좋', '다']]

In [19]:
new_ans_corpus[:5]

[['하루', '가', '또', '가', '네요', '.'],
 ['위로', '해', '드립니다', '.'],
 ['잘', '모르', '고', '있', '을', '수', '도', '있', '어요', '.'],
 ['그', '사람', '도', '그럴', '거', '예요', '.'],
 ['혼자', '를', '즐기', '세요', '.']]

## 5. 데이터 벡터화

- 타겟 데이터인 'ans_corpus'에 \<start\> 토큰과 \<end\> 토큰이 추가되지 않은 상태이니 이를 먼저 해결한 후 벡터화를 진행
- 구축한 'ans_corpus'는 'list' 형태이기 때문에 아주 쉽게 이를 해결할 수 있음

In [20]:
sample_data = ["12", "시", "땡", "!"]

print(["<start>"] + sample_data + ["<end>"])

['<start>', '12', '시', '땡', '!', '<end>']


In [22]:
temp = []

for corpus in ans_corpus:
    temp.append(["<start>"] + corpus + ["<end>"])

In [23]:
ans_corpus = temp

In [24]:
ans_corpus[:5]

[['<start>', '하루', '가', '또', '가', '네요', '.', '<end>'],
 ['<start>', '위로', '해', '드립니다', '.', '<end>'],
 ['<start>', '여행', '은', '언제나', '좋', '죠', '.', '<end>'],
 ['<start>', '눈살', '이', '찌푸려', '지', '죠', '.', '<end>'],
 ['<start>', '다시', '새로', '사', '는', '게', '마음', '편해요', '.', '<end>']]

In [25]:
total_data = que_corpus + ans_corpus
len(total_data)

15274

In [26]:
words = np.concatenate(total_data).tolist()
counter = Counter(words)
counter = counter.most_common(30000-2)
vocab = ['<pad>', '<unk>'] + [key for key, _ in counter]
word_to_index = {word:index for index, word in enumerate(vocab)}
index_to_word = {index:word for word, index in word_to_index.items()}

In [27]:
word_to_index

{'<pad>': 0,
 '<unk>': 1,
 '.': 2,
 '<start>': 3,
 '<end>': 4,
 '이': 5,
 '는': 6,
 '하': 7,
 '을': 8,
 '가': 9,
 '좋': 10,
 '세요': 11,
 '고': 12,
 '어': 13,
 '거': 14,
 '있': 15,
 '은': 16,
 '해': 17,
 '보': 18,
 '지': 19,
 '?': 20,
 '나': 21,
 '아': 22,
 '도': 23,
 '게': 24,
 '겠': 25,
 '에': 26,
 '사람': 27,
 '예요': 28,
 '사랑': 29,
 '어요': 30,
 '를': 31,
 '같': 32,
 '한': 33,
 '죠': 34,
 '다': 35,
 '네': 36,
 '면': 37,
 '수': 38,
 '네요': 39,
 '의': 40,
 '안': 41,
 '것': 42,
 '없': 43,
 '싶': 44,
 '친구': 45,
 '는데': 46,
 '생각': 47,
 '봐요': 48,
 '않': 49,
 '아요': 50,
 '마음': 51,
 '말': 52,
 '할': 53,
 '너무': 54,
 '되': 55,
 '이별': 56,
 '잘': 57,
 '주': 58,
 '했': 59,
 '었': 60,
 '내': 61,
 '남자': 62,
 '기': 63,
 '연락': 64,
 '만': 65,
 '더': 66,
 '일': 67,
 '들': 68,
 '여자': 69,
 '힘들': 70,
 '남': 71,
 '해요': 72,
 '시간': 73,
 '썸': 74,
 '짝': 75,
 '길': 76,
 '많이': 77,
 '으로': 78,
 '한테': 79,
 '았': 80,
 '으면': 81,
 '건': 82,
 '때': 83,
 '에서': 84,
 '에요': 85,
 '좀': 86,
 '요': 87,
 '야': 88,
 '알': 89,
 '그': 90,
 '만나': 91,
 '많': 92,
 '에게': 93,
 '로': 94,
 '받': 95,
 '을까

In [28]:
def get_encoded_sentence(sentence, word_to_index):
    return [word_to_index[word] if word in word_to_index else word_to_index['<unk>'] for word in sentence]

In [29]:
def get_decoded_sentence(encoded_sentence, index_to_word):
    return ' '.join(index_to_word[index] if index in index_to_word else '<unk>' for index in encoded_sentence[1:])

In [30]:
def vectorize(corpus, word_to_index):
    data = []
    for sen in corpus:
        sen = get_encoded_sentence(sen, word_to_index)
        data.append(sen)
    return data

In [31]:
que_train = vectorize(que_corpus, word_to_index)
ans_train = vectorize(ans_corpus, word_to_index)

In [32]:
enc_train = keras.preprocessing.sequence.pad_sequences(que_train, padding='pre', maxlen=20)
dec_train = keras.preprocessing.sequence.pad_sequences(ans_train, padding='pre', maxlen=20)

In [33]:
enc_train[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0, 2032,  204, 2554,  107], dtype=int32)

In [34]:
dec_train[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,
       269,   9, 136,   9,  39,   2,   4], dtype=int32)

    - 1. 위 소스를 참고하여 타겟 데이터 전체에 '**\<start**\>' 토큰과 '**\<end\>**' 토큰을 추가
<br>

- 챗봇 훈련 데이터의 가장 큰 특징 중 하나라고 하자면 바로 **소스 데이터와 타겟 데이터가 같은 언어를 사용한다는 것**
- 앞서 배운 것처럼 이는 Embedding 층을 공유했을 때 많은 이점을 얻을 수 있음
<br>

    - 2. 특수 토큰을 더함으로써 '**ans_corpus**' 또한 완성이 되었으니, '**que_corpus**'와 결합하여 **전체 데이터에 대한 단어 사전을 구축**하고 **벡터화하여** '**enc_train**'과 '**dec_train**'을 얻기


## 6. 훈련하기

- 앞서 번역 모델을 훈련하며 정의한 'Transformer'를 그대로 사용하면 됨
- 대신 데이터의 크기가 작으니 하이퍼파라미터를 튜닝해야 과적합 피할 수 있음
- 모델을 훈련하고 아래 예문에 대한 답변을 생성
- **가장 멋진 답변**과 **모델의 하이퍼파라미터**를 제출하면 됨

### 6-1. Positional Encoding

In [35]:
def positional_encoding(pos, d_model):
    def cal_angle(position, i):
        return position / np.power(10000, int(i) / d_model)

    def get_posi_angle_vec(position):
        return [cal_angle(position, i) for i in range(d_model)]

    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(pos)])

    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])

    return sinusoid_table

### 6-2. 마스크 생성

In [36]:
def generate_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

def generate_causality_mask(src_len, tgt_len):
    mask = 1 - np.cumsum(np.eye(src_len, tgt_len), 0)
    return tf.cast(mask, tf.float32)

def generate_masks(src, tgt):
    enc_mask = generate_padding_mask(src)
    dec_mask = generate_padding_mask(tgt)

    dec_causality_mask = generate_causality_mask(tgt.shape[1], tgt.shape[1])
    dec_mask = tf.maximum(dec_mask, dec_causality_mask)

    dec_enc_causality_mask = generate_causality_mask(tgt.shape[1], src.shape[1])
    dec_enc_mask = tf.maximum(enc_mask, dec_enc_causality_mask)

    return enc_mask, dec_enc_mask, dec_mask

### 6-3. Multi-head Attention

In [37]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        self.depth = d_model // self.num_heads

        self.W_q = tf.keras.layers.Dense(d_model)
        self.W_k = tf.keras.layers.Dense(d_model)
        self.W_v = tf.keras.layers.Dense(d_model)

        self.linear = tf.keras.layers.Dense(d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask):
        d_k = tf.cast(K.shape[-1], tf.float32)
        QK = tf.matmul(Q, K, transpose_b=True)

        scaled_qk = QK / tf.math.sqrt(d_k)

        if mask is not None: scaled_qk += (mask * -1e9)  

        attentions = tf.nn.softmax(scaled_qk, axis=-1)
        out = tf.matmul(attentions, V)

        return out, attentions


    def split_heads(self, x):
        bsz = x.shape[0]
        split_x = tf.reshape(x, (bsz, -1, self.num_heads, self.depth))
        split_x = tf.transpose(split_x, perm=[0, 2, 1, 3])

        return split_x

    def combine_heads(self, x):
        bsz = x.shape[0]
        combined_x = tf.transpose(x, perm=[0, 2, 1, 3])
        combined_x = tf.reshape(combined_x, (bsz, -1, self.d_model))

        return combined_x


    def call(self, Q, K, V, mask):
        WQ = self.W_q(Q)
        WK = self.W_k(K)
        WV = self.W_v(V)

        WQ_splits = self.split_heads(WQ)
        WK_splits = self.split_heads(WK)
        WV_splits = self.split_heads(WV)

        out, attention_weights = self.scaled_dot_product_attention(
            WQ_splits, WK_splits, WV_splits, mask)

        out = self.combine_heads(out)
        out = self.linear(out)

        return out, attention_weights

### 6-4. Position-wise Feed Forward Network

In [38]:
class PoswiseFeedForwardNet(tf.keras.layers.Layer):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff

        self.fc1 = tf.keras.layers.Dense(d_ff, activation='relu')
        self.fc2 = tf.keras.layers.Dense(d_model)

    def call(self, x):
        out = self.fc1(x)
        out = self.fc2(out)

        return out

### 6-5. Encoder Layer

In [39]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()

        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)

    def call(self, x, mask):

        """
        Multi-Head Attention
        """
        residual = x
        out = self.norm_1(x)
        out, enc_attn = self.enc_self_attn(out, out, out, mask)
        out = self.do(out)
        out += residual

        """
        Position-Wise Feed Forward Network
        """
        residual = out
        out = self.norm_2(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual

        return out, enc_attn

### 6-6. Decoder Layer

In [40]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()

        self.dec_self_attn = MultiHeadAttention(d_model, num_heads)
        self.enc_dec_attn = MultiHeadAttention(d_model, num_heads)

        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)

    def call(self, x, enc_out, causality_mask, padding_mask):

        """
        Masked Multi-Head Attention
        """
        residual = x
        out = self.norm_1(x)
        out, dec_attn = self.dec_self_attn(out, out, out, padding_mask)
        out = self.do(out)
        out += residual

        """
        Multi-Head Attention
        """
        residual = out
        out = self.norm_2(out)
        out, dec_enc_attn = self.dec_self_attn(out, enc_out, enc_out, causality_mask)
        out = self.do(out)
        out += residual

        """
        Position-Wise Feed Forward Network
        """
        residual = out
        out = self.norm_3(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual

        return out, dec_attn, dec_enc_attn

### 6-7. Encoder

In [41]:
class Encoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.enc_layers = [EncoderLayer(d_model, n_heads, d_ff, dropout) 
                        for _ in range(n_layers)]

        self.do = tf.keras.layers.Dropout(dropout)

    def call(self, x, mask):
        out = x

        enc_attns = list()
        for i in range(self.n_layers):
            out, enc_attn = self.enc_layers[i](out, mask)
            enc_attns.append(enc_attn)

        return out, enc_attns

### 6-8. Decoder

In [42]:
class Decoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.dec_layers = [DecoderLayer(d_model, n_heads, d_ff, dropout) 
                            for _ in range(n_layers)]


    def call(self, x, enc_out, causality_mask, padding_mask):
        out = x

        dec_attns = list()
        dec_enc_attns = list()
        for i in range(self.n_layers):
            out, dec_attn, dec_enc_attn = \
            self.dec_layers[i](out, enc_out, causality_mask, padding_mask)

            dec_attns.append(dec_attn)
            dec_enc_attns.append(dec_enc_attn)

        return out, dec_attns, dec_enc_attns

### 6-9. Transformer 전체 모델 조립

In [43]:
class Transformer(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    src_vocab_size,
                    tgt_vocab_size,
                    pos_len,
                    dropout=0.2,
                    shared_fc=True,
                    shared_emb=False):
        super(Transformer, self).__init__()

        self.d_model = tf.cast(d_model, tf.float32)

        if shared_emb:
            self.enc_emb = self.dec_emb = \
            tf.keras.layers.Embedding(src_vocab_size, d_model)
        else:
            self.enc_emb = tf.keras.layers.Embedding(src_vocab_size, d_model)
            self.dec_emb = tf.keras.layers.Embedding(tgt_vocab_size, d_model)

        self.pos_encoding = positional_encoding(pos_len, d_model)
        self.do = tf.keras.layers.Dropout(dropout)

        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)

        self.fc = tf.keras.layers.Dense(tgt_vocab_size)

        self.shared_fc = shared_fc

        if shared_fc:
            self.fc.set_weights(tf.transpose(self.dec_emb.weights))

    def embedding(self, emb, x):
        seq_len = x.shape[1]

        out = emb(x)

        if self.shared_fc: out *= tf.math.sqrt(self.d_model)

        out += self.pos_encoding[np.newaxis, ...][:, :seq_len, :]
        out = self.do(out)

        return out


    def call(self, enc_in, dec_in, enc_mask, causality_mask, dec_mask):
        enc_in = self.embedding(self.enc_emb, enc_in)
        dec_in = self.embedding(self.dec_emb, dec_in)

        enc_out, enc_attns = self.encoder(enc_in, enc_mask)

        dec_out, dec_attns, dec_enc_attns = \
        self.decoder(dec_in, enc_out, causality_mask, dec_mask)

        logits = self.fc(dec_out)

        return logits, enc_attns, dec_attns, dec_enc_attns

### 6-10. 모델 인스턴스 생성

In [44]:
VOCAB_SIZE = 30000

transformer = Transformer(
    n_layers=5,
    d_model=512,
    n_heads=8,
    d_ff=2048,
    src_vocab_size=VOCAB_SIZE,
    tgt_vocab_size=VOCAB_SIZE,
    pos_len=200,
    dropout=0.3,
    shared_fc=True,
    shared_emb=True)

d_model = 512

### 6-11. Learning Rate Scheduler

In [45]:
class LearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(LearningRateScheduler, self).__init__()

        self.d_model = d_model
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = step ** -0.5
        arg2 = step * (self.warmup_steps ** -1.5)

        return (self.d_model ** -0.5) * tf.math.minimum(arg1, arg2)

### 6-12. Learing Rate & Optimizer

In [46]:
learning_rate = LearningRateScheduler(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate,
                                        beta_1=0.9,
                                        beta_2=0.98, 
                                        epsilon=1e-9)

### 6-13. Loss Function 정의

In [47]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

### 6-14. Train Step 정의

In [48]:
@tf.function()
def train_step(src, tgt, model, optimizer):
    tgt_in = tgt[:, :-1]
    gold = tgt[:, 1:]

    enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt_in)

    with tf.GradientTape() as tape:
        predictions, enc_attns, dec_attns, dec_enc_attns = \
        model(src, tgt_in, enc_mask, dec_enc_mask, dec_mask)
        loss = loss_function(gold, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss, enc_attns, dec_attns, dec_enc_attns

### 6-15. 훈련

In [49]:
from tqdm import tqdm_notebook 

BATCH_SIZE = 128
EPOCHS = 30

for epoch in range(EPOCHS):
    total_loss = 0

    idx_list = list(range(0, enc_train.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm_notebook(idx_list)

    for (batch, idx) in enumerate(t):
        batch_loss, enc_attns, dec_attns, dec_enc_attns = \
        train_step(enc_train[idx:idx+BATCH_SIZE],
                    dec_train[idx:idx+BATCH_SIZE],
                    transformer,
                    optimizer)

        total_loss += batch_loss

        t.set_description_str('Epoch %2d' % (epoch + 1))
        t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1)))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

...

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

## 7. 성능 측정하기

- 챗봇의 경우, 올바른 대답을 하는지가 중요한 평가 지표
- 올바른 답변을 하는지 눈으로 확인할 수 있겠지만, 많은 데이터의 경우는 모든 결과를 확인할 수 없을 것
- 주어진 질문에 적절한 답변을 하는지 확인하고, BLEU Score를 계산하는 'calculate_bleu()'함수도 적용해 보기

In [50]:
def evaluate(sentence, model):
    mecab = Mecab()
    
    sentence = preprocess_sentence(sentence)
    pieces = mecab.morphs(sentence)
    
    tokens = []
    for sen in pieces:
        sen= get_encoded_sentence(sen, word_to_index)
        tokens.append(sen)
    
    _input = tf.keras.preprocessing.sequence.pad_sequences(tokens,
                                                        value=word_to_index["<pad>"],
                                                        padding='pre',
                                                        maxlen=20)
    
    ids = []
    output = tf.expand_dims([word_to_index["<start>"]], 0)
    for i in range(dec_train.shape[-1]):
        enc_padding_mask, combined_mask, dec_padding_mask = \
        generate_masks(_input, output)

        predictions, enc_attns, dec_attns, dec_enc_attns =\
        model(_input, 
              output,
              enc_padding_mask,
              combined_mask,
              dec_padding_mask)

        predicted_id = \
        tf.argmax(tf.math.softmax(predictions, axis=-1)[0, -1]).numpy().item()

        if word_to_index["<end>"] == predicted_id:
            result = get_decoded_sentence(ids, index_to_word)
            return pieces, result, enc_attns, dec_attns, dec_enc_attns

        ids.append(predicted_id)
        output = tf.concat([output, tf.expand_dims([predicted_id], 0)], axis=-1)

    result = get_decoded_sentence(ids, index_to_word)

    return pieces, result, enc_attns, dec_attns, dec_enc_attns

def translate(sentence, model):
    pieces, result, enc_attns, dec_attns, dec_enc_attns = \
    evaluate(sentence, model)

    return result

In [51]:
samples = ["지루하다, 놀러가고 싶어.", "오늘 일찍 일어났더니 피곤하다.", "간만에 여자친구랑 데이트 하기로 했어.", "집에 있는다는 소리야."]

In [52]:
for sample in samples:
    print('sample : ', sample)
    print('Translations : ', translate(sample, transformer))

sample :  지루하다, 놀러가고 싶어.
Translations :  가 있 지만 다른 거 예요 . 제 에 있 지만 제 가 될 거 예요 .
sample :  오늘 일찍 일어났더니 피곤하다.
Translations :  씩 아프 지 었 나 었 나 었 나 었 나 었 었 었 나 봐요 .
sample :  간만에 여자친구랑 데이트 하기로 했어.
Translations :  일 이 었 겠 지만 큰 큰 큰 큰 큰 큰 큰 큰 모두 되 겠 어요 .
sample :  집에 있는다는 소리야.
Translations :  고 싶 기 도 요 . 같이 요 . 같이 요 . 같이 되 기 요 .
